In [1]:
import pyspark.sql.functions as F
from credentials import MY_CREDENTIALS
from data_location import DELTA_LOCATION

from spark_bi.spark import FutPathlingContext

pc = FutPathlingContext.create(
    app_name="example-spark-app", hadoop_config=MY_CREDENTIALS.to_hadoop_config()
)
delta_lake = pc.read.delta(DELTA_LOCATION)

:: loading settings :: url = jar:file:/Users/mabe/Git/spark-bi/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/mabe/.ivy2.5.2/cache
The jars for the packages stored in: /Users/mabe/.ivy2.5.2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a6ea51fa-a5ab-4e1f-99f4-fef558b24500;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;9.1.0 in local-m2-cache
	found io.delta#delta-spark_2.13;4.0.0 in local-m2-cache
	found io.delta#delta-storage;4.0.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.13.1 in local-m2-cache
:: resolution report :: resolve 90ms :: artifacts dl 6ms
	:: modules in use:
	au.csiro.pathling#library-runtime;9.1.0 from local-m2-cache in [default]
	io.delta#delta-spark_2.13;4.0.0 from local-m2-cache in [default]
	io.delta#delta-storage;4.0.0 fr

In [2]:
careteams = delta_lake.view(
    resource="CareTeam",
    select=[
        {
            "column": [
                {"name": "careteam_id", "path": "getResourceKey()"},
                {"name": "org_id", "path": "managingOrganization.first().getReferenceKey()"},
            ]
        }
    ],
)
careteams.filter(F.col("org_id").isNotNull()).head(5)

[Row(careteam_id='CareTeam/3000148060', org_id='Organization/3000038806'),
 Row(careteam_id='CareTeam/3000148061', org_id='Organization/3000029719')]

In [3]:
organizations = delta_lake.view(
    resource="Organization",
    select=[
        {
            "column": [
                {"name": "org_id", "path": "getResourceKey()"},
                {
                    "name": "municipality_code",
                    "path": "extension('http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-organization-municipalityCode').valueString",
                },
            ]
        }
    ],
)
organizations.filter(F.col("municipality_code").isNotNull()).head(5)

[Row(org_id='Organization/3000000064', municipality_code='0787'),
 Row(org_id='Organization/3000000069', municipality_code='0265'),
 Row(org_id='Organization/3000000072', municipality_code='0173'),
 Row(org_id='Organization/3000000088', municipality_code='0360'),
 Row(org_id='Organization/3000000090', municipality_code='0787')]

In [4]:
joined = careteams.join(organizations, careteams.org_id == organizations.org_id, how="left")
joined.head(5)

[Row(careteam_id='CareTeam/3000000001', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000002', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000005', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000006', org_id=None, org_id=None, municipality_code=None),
 Row(careteam_id='CareTeam/3000000007', org_id=None, org_id=None, municipality_code=None)]

In [5]:
joined.filter(F.col("municipality_code").isNotNull()).head(5)

[Row(careteam_id='CareTeam/3000148060', org_id='Organization/3000038806', org_id='Organization/3000038806', municipality_code='0851'),
 Row(careteam_id='CareTeam/3000148061', org_id='Organization/3000029719', org_id='Organization/3000029719', municipality_code='0849')]

På TRIFORKs testmiljø er der kun 2 careteams der er tilknyttet en kommune. De kan vises i en oversigt:

In [6]:
joined.groupBy("municipality_code").agg(
    F.countDistinct("careteam_id").alias("n_careteams")
).orderBy("n_careteams", ascending=False).toPandas()

,municipality_code,n_careteams
0,None,31
1,0851,1
2,0849,1
